In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from matplotlib import cm, colors
import matplotlib.pyplot as plt

import numpy as np

import dataUtils as du
import myModels as mm

import tempfile
import os

import tensorflow_model_optimization as tfmot


%load_ext tensorboard

2024-02-14 21:46:29.926584: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 21:46:29.928935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-14 21:46:29.962342: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 21:46:29.962395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 21:46:29.962437: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
NUMEPOCHS = 100
BATCHSIZE = 128
PHASEMAX = 100
PERCENTPILEUP = 0.5
AUGMENTATION = 8
TRACELENGTH = 250
INPUTSIZE = TRACELENGTH-4*AUGMENTATION
VALIDATIONSPLIT = 0.4

In [3]:
# Load the data
x_train, x_test, y_train, y_test = du.GetDataSet(type="phase",
                                                 fname="DataSmallFloat.root",
                                                 tname="OutputTree")


Train size:  269082
Test size:  269083


In [ ]:
# Phase
- Arch prune starting from bigger model (current mse: 1.7)
- Weight prune starting from smaller model (current mse: 1.7) 

* goal is mse of 1.0 (2ns)

# Amp
- Arch prune starting from bigger model (current mse: 0.11)

* goal 0.05 

# Pileup
- No early stopping (current acc 0.97)

* goal 0.99 

- Data 


In [4]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)


def evaluate_model(interpreter, x_test, y_test):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on ever y image in the "test" dataset.
    prediction = []
    for i, test_trace in enumerate(x_test):
        if i % 1000 == 0:
            print('Evaluated on {n} results so far.'.format(n=i))
        
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_trace = np.expand_dims(test_trace, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_trace)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        y_pred = np.argmax(output()[0])
        prediction.append(y_pred)

    print('\n')

    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction = np.array(prediction)
    mse = np.mean((prediction - y_test)**2)
    return mse



def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')


In [5]:
# get baseline model
model = mm.PhaseNet(INPUTSIZE)
model.summary()

# early stopping callback based on validation loss
early_stopping = EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

# compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanSquaredError()],
)

# train the model
history = model.fit(
    x_train,
    y_train,
    batch_size=BATCHSIZE,
    epochs=NUMEPOCHS,
    validation_split=VALIDATIONSPLIT,
    callbacks=[early_stopping],
)

# evaluate the model
mse_baseline = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Mean Squared Error: {mse_baseline[0]}")

# Save the model
_ , keras_file = tempfile.mkstemp('.h5')
model.save(keras_file, include_optimizer=False)
print(f"Saved model to: {keras_file}")

#pruning
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
num_samples = x_train.shape[0] * (1-VALIDATIONSPLIT)
end_step = np.ceil(num_samples / BATCHSIZE).astype(np.int32) * NUMEPOCHS

pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                 final_sparsity=0.80,
                                                                 begin_step=0,
                                                                 end_step=end_step)
    }

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_for_pruning.summary()

logdir = tempfile.mkdtemp()

callbacks = [ tfmot.sparsity.keras.UpdatePruningStep(),
                tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)]

pruned_history = model_for_pruning.fit(x_train, y_train, 
          epochs=NUMEPOCHS, 
          batch_size=BATCHSIZE,
          validation_split=VALIDATIONSPLIT,
            callbacks=callbacks)

pruned_mse, _ = model_for_pruning.evaluate(x_test, y_test, verbose=0)

print(f"Test MSE after pruning: {pruned_mse}")

%tensorboard --logdir={logdir}

_, keras_pruned_file = tempfile.mkstemp('.h5')
# tf.keras.models.save_model(model_for_pruning, keras_pruned_file, include_optimizer=False)
model_for_pruning.save(keras_pruned_file, include_optimizer=False)
print(f"Saved pruned model to: {keras_pruned_file}")

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_pruning)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print(f"Saved pruned TFLite model to: {pruned_tflite_file}")

    
# size of the models

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(keras_pruned_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(tflite_quant_model)

print(f"Saved quantized and pruned TFLite model to: {quantized_and_pruned_tflite_file}")
print("Size of gzipped quantized and pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

# compare the models

interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_mse = evaluate_model(interpreter, x_test, y_test)

print('Baseline test MSE:', mse_baseline[0])
print('Pruned and quantized TFLite test MSE:', test_mse)
print('Pruned and quantized TFLite test MSE:', pruned_mse)

Model: "PhaseNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 209, 64)           704       
                                                                 
 dropout1 (Dropout)          (None, 209, 64)           0         
                                                                 
 conv2 (Conv1D)              (None, 209, 64)           4160      
                                                                 
 dropout2 (Dropout)          (None, 209, 64)           0         
                                                                 
 conv3 (Conv1D)              (None, 209, 64)           8256      
                                                                 
 dropout3 (Dropout)          (None, 209, 64)           0         
                                                                 
 flat1 (Flatten)             (None, 13376)             0  

/home/tmengel/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "PhaseNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv1   (None, 209, 64)           1346      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_dropou  (None, 209, 64)           1         
 t1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2   (None, 209, 64)           8258      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_dropou  (None, 209, 64)           1         
 t2 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv3   (None, 209, 64)           164

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-02-15 02:44:00.382534: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 02:44:00.388995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 02:44:00.461562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 02:44:00.461620: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 02:44:00.461945: E tensorflow/compiler/xla/stream_executor/cuda

/home/tmengel/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

INFO:tensorflow:Assets written to: /tmp/tmp0mmr2lme/assets
2024-02-15 02:44:06.505315: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-15 02:44:06.505385: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-15 02:44:06.506434: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0mmr2lme
2024-02-15 02:44:06.511199: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-15 02:44:06.511218: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0mmr2lme
2024-02-15 02:44:06.521314: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-02-15 02:44:06.524955: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-15 02:44:06.613194: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

Saved pruned TFLite model to: /tmp/tmp0ah2f0gd.tflite
Size of gzipped baseline Keras model: 6482852.00 bytes
Size of gzipped pruned Keras model: 2372228.00 bytes
Size of gzipped pruned TFlite model: 2370219.00 bytes
INFO:tensorflow:Assets written to: /tmp/tmpx8wl_6ag/assets


INFO:tensorflow:Assets written to: /tmp/tmpx8wl_6ag/assets
2024-02-15 02:44:09.096390: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-15 02:44:09.096441: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-15 02:44:09.096601: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpx8wl_6ag
2024-02-15 02:44:09.098038: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-15 02:44:09.098056: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpx8wl_6ag
2024-02-15 02:44:09.101752: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-15 02:44:09.152835: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpx8wl_6ag
2024-02-15 02:44:09.168154: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

Saved quantized and pruned TFLite model to: /tmp/tmpnw8qslgq.tflite
Size of gzipped quantized and pruned TFlite model: 339858.00 bytes
Evaluated on 0 results so far.


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 3 for input 0.

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}


model = mm.PhaseNetPrunable(INPUTSIZE, pruning_params_2_by_4, pruning_params_sparsity_0_5)

model.summary()


model.compile(optimizer='adam',
                loss='mse',
                metrics=['mse'])

x_train, x_test, y_train, y_test = du.GetDataSet(type="phase",
                                                 fname="DataSmallFloat.root",
                                                 tname="OutputTree")

callbaks = [tfmot.sparsity.keras.UpdatePruningStep()]
history = model.fit(x_train, y_train,
                    epochs=NUMEPOCHS,
                    batch_size=BATCHSIZE,
                    validation_split=VALIDATIONSPLIT,
                    callbacks=callbaks)

pruned_mse, _ = model.evaluate(x_test, y_test, verbose=0)
print(f"Test MSE after pruning: {pruned_mse}")

model = tfmot.sparsity.keras.strip_pruning(model)


converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

  # Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file, experimental_preserve_all_tensors=True)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

print(f"Shape of Dense layer is {tensor_data.shape}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24
subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)


plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
op_details = interpreter._get_ops_details()
op_name = 'CONV_2D'
op_detail = [x for x in op_details if op_name in x["op_name"]]
tensor_data = interpreter.tensor(op_detail[1]["inputs"][1])()
print(f"Shape of the weight tensor is {tensor_data.shape}")


weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()